In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm

In [2]:
files = ['bproperty_bashundhara_RA.csv',
         'bproperty_dhanmondi.csv',
         'bproperty_gulshan.csv',
         'bproperty_mirpur.csv',
         'bproperty_uttara.csv' ]

In [3]:
dfs = []
for file in files:
    dfs.append(pd.read_csv(file))
df = pd.concat(dfs)

In [4]:
df

,dimensions,num_beds,num_baths,house_type,city,location,area_block,description,url,price
0,"2,055 sqft",4,4,Apartment,Dhaka,Bashundhara R-A,Block D,2055 Sq Ft Cozy Flat For Rent In Bashundhara R...,/en/property/details-5237946.html,35000.0
1,"1,911 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block A,Emanate Your Knack For Gardening By Renting Th...,/en/property/details-3875341.html,45000.0
2,"1,200 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block K,"To Secure Your Better State Of Living, Conside...",/en/property/details-5182996.html,20000.0
3,"1,910 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block A,For Rental Purpose 1910 Sq Ft Commendable Desi...,/en/property/details-3875358.html,45000.0
4,800 sqft,2,2,Apartment,Dhaka,Bashundhara R-A,Block G,Bright And Cozy Apartment Featuring 800 Sq Ft ...,/en/property/details-3651253.html,18000.0
...,...,...,...,...,...,...,...,...,...,...
2726,"2,500 sqft",3,4,Apartment,Dhaka,Uttara,Sector 4,For You 2500 Sq Ft Flat Is Now For Rent Near T...,/en/property/details-1701632.html,50000.0
2727,"1,600 sqft",3,3,Apartment,Dhaka,Uttara,Sector 6,A noteworthy Residence up for rent in Uttara a...,/en/property/details-1701287.html,25000.0
2728,900 sqft,2,2,Apartment,Dhaka,Uttara,Sector 14,A 900 SQ FT vacant apartment is ready to be re...,/en/property/details-1667106.html,16000.0
2729,"1,200 sqft",3,3,Apartment,Dhaka,Uttara,Sector 5,Residential Flat Of 1200 Sq Ft Is Available Fo...,/en/property/details-1665314.html,25000.0


In [5]:
df['location'].value_counts()

Mirpur             7935
Uttara             2731
Bashundhara R-A    1542
Dhanmondi           981
Gulshan             226
Name: location, dtype: int64

In [6]:
df['area_block'].value_counts()

Section 12                  1300
Pallabi                      853
Section 10                   840
Section 1                    711
Pirerbag                     437
                            ... 
Borobag                        1
North Karful Road              1
South Paikpara Main Road       1
Tolarbag                       1
Baten Nagar                    1
Name: area_block, Length: 122, dtype: int64

In [21]:
url = 'https://www.bproperty.com/en/property/details-3376701.html'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
text = response.text
soup = BeautifulSoup(text, 'lxml')

In [22]:
features = soup.find_all("span", class_='_005a682a')

In [23]:
features

[<span class="_005a682a">Parking Spaces<!-- -->: 2</span>,
 <span class="_005a682a">Floor Level</span>,
 <span class="_005a682a">Balcony or Terrace</span>,
 <span class="_005a682a">View</span>,
 <span class="_005a682a">Flooring</span>,
 <span class="_005a682a">Elevators in Building<!-- -->: 1</span>,
 <span class="_005a682a">24 Hours Concierge</span>,
 <span class="_005a682a">Lobby in Building</span>,
 <span class="_005a682a">Service Elevators</span>,
 <span class="_005a682a">Elevators in Building<!-- -->: 1</span>,
 <span class="_005a682a">Parking Spaces<!-- -->: 2</span>,
 <span class="_005a682a">24 Hours Concierge</span>,
 <span class="_005a682a">Floor Level</span>,
 <span class="_005a682a">Balcony or Terrace</span>,
 <span class="_005a682a">Lobby in Building</span>,
 <span class="_005a682a">Service Elevators</span>,
 <span class="_005a682a">View</span>,
 <span class="_005a682a">Electricity Backup</span>,
 <span class="_005a682a">Flooring</span>,
 <span class="_005a682a">Intercom</s

In [24]:
dicy = {}
for i in tqdm(range(len(features))):
    features[i] = str(features[i]).replace('<span class="_005a682a">', '').replace('</span>', '').replace('<!-- -->', '').replace(': ', ':')
    try:
        key, value = features[i].split(':')
    except:
        key = features[i]
        value = 'Yes'
    dicy[key] = value
features = list(set(features))

  0%|          | 0/23 [00:00<?, ?it/s]

In [25]:
dicy

{'Parking Spaces': '2',
 'Floor Level': 'Yes',
 'Balcony or Terrace': 'Yes',
 'View': 'Yes',
 'Flooring': 'Yes',
 'Elevators in Building': '1',
 '24 Hours Concierge': 'Yes',
 'Lobby in Building': 'Yes',
 'Service Elevators': 'Yes',
 'Electricity Backup': 'Yes',
 'Intercom': 'Yes',
 'CCTV Security': 'Yes',
 'Maintenance Staff': 'Yes',
 'Cleaning Services': 'Yes'}